In [3]:
# Deleting files that we do not need, files generated from a previous run
import os
import glob

extension_list=["*inp*","*out","*ener","*rest*","*Hess*","*REST*","*.xyz","*.pdb"]
for name in extension_list:
    
    filelist=glob.glob(name)
    for file in filelist:
        os.remove(file)


# ### Loading modules




import mbuild as mb 
import numpy as np
import unyt as u

from cp2kmdpy.molecule_optimization import Molecule_optimization # for single molecule optimization
from cp2kmdpy.md import MD # for running MD
from cp2kmdpy import runners
import setter

In [4]:
!ls

MDsimulation.ipynb  __pycache__  setter.py


In [5]:

class O2(mb.Compound): # this class builds an oxygen molecule with a bond-length given in the oxygen2 x coor (nm)
    def __init__(self):
        super(O2, self).__init__()
        
        oxygen1= mb.Particle(pos=[0.0, 0.0, 0.0], name='O')
        oxygen2= mb.Particle(pos=[0.15, 0.0, 0.0], name='O')
        self.add([oxygen2,oxygen1])
        self.add_bond((oxygen2,oxygen1))


In [6]:
molecule=O2();
box=mb.box.Box(lengths=[1,1,1])

In [12]:
oxygen_optimization=Molecule_optimization(molecule=molecule,basis_set={'O':'DZVP-MOLOPT-GTH'},box=box,cutoff=300,functional='PBE',periodicity='NONE',n_iter=1,input_filename='rand.inp')

In [13]:
oxygen_optimization.optimization_initialization()

scf_tolerance not specified, set as 1e-6
basis_set_filename not defined, set as BASIS_MOLOPT
potential_filename not specified, set as GTH_POTENTIALS
fixed_list not specified, set as 1
output_filename not specified, set as O2_optimization_output.out
Output position filename is O2_opt-pos-1.xyz
You can change default settings in setter.single_molecule_opt_file


In [14]:
setter.single_molecule_opt_files(oxygen_optimization)

Initial structure saved as O2_unoptimized_coord.xyz
The box should be cubic for non-periodic calculations and the box must be around 15 times the size of the molecule when periodicity is NONE
Molecule optimization file saved as rand.inp


In [15]:


oxygen_optimization.run_optimization()

Input file name given to runner is rand.inp
Output file name  is O2_optimization_output.out


In [16]:
optimized_oxygen=oxygen_optimization.return_optimized_molecule()

In [17]:
np.linalg.norm(optimized_oxygen.xyz[1]-optimized_oxygen.xyz[0])

0.15708195023131846

In [18]:
q=MD(molecules=[optimized_oxygen],box=box,cutoff=200,functional='PBE',basis_set={'O':'DZVP-MOLOPT-GTH'},periodicity='XYZ',n_molecules=2,traj_type='PDB',seed=1)


In [19]:
q.temperature=273.15*u.K
q.ensemble='NVT'

In [20]:
q.md_initialization()

project_name not specified, set as sample_project
scf_tolerance not specified, set as 1e-6
basis_set_filename not defined, set as BASIS_MOLOPT
potential_filename not specified, set as GTH_POTENTIALS
simulation_time not specified, set as 1 ps
time_step not specified, time_step set as 1 fs as the lighest element has mass 15.9994 au
input_filename not specified, set as sample_project_md_input.inp
output_filename not specified, set as sample_project_md_output.out
Output position filename is sample_project-pos-1.xyz
You can change default settings in setter.md_files


In [21]:
setter.md_files(q)

MD initial structure saved as sample_project.xyz
MD input file saved as sample_project_md_input.inp


In [22]:
runners.run_md('sample_project_md_input.inp','sample_project_md_output.out' )

KeyboardInterrupt: 

In [ ]:
!pwd